In [ ]:
import os
from tqdm.auto import tqdm

import numpy as np
import torch
from torchvision import models
from torch.utils.data import DataLoader
from torchvision import transforms

import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.style.use('seaborn-v0_8-colorblind')


import data_utils
from autoattack import AutoAttack
# (https://github.com/fra31/auto-attack/blob/master/autoattack/autoattack.py)


In [ ]:
##load model and defaalt image transform
model, preprocess0 = data_utils.get_target_model(target_name='resnet50', device='cuda', weights='default')
preprocess0

In [ ]:
##transforms
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])

normalize = transforms.Normalize(mean=preprocess0.mean, std=preprocess0.std)

def denormalize(x):
    std = torch.tensor(preprocess0.std).view(3,1,1)
    mean = torch.tensor(preprocess0.mean).view(3,1,1)
    x = x * std + mean # de-normalize
    return x

to_pil = transforms.ToPILImage()


## model forward pass
def forward_pass(img):
    img = normalize(img)
    return model(img)

In [ ]:
##data, loader
dataset = data_utils.get_data('imagenet_val', preprocess)
loader = DataLoader(dataset, batch_size=8)

In [ ]:
# ## test for correctness

# img, target = next(iter(loader))
# img, target = img.cuda(), target.cuda()
# r = img.max()-img.min() # r=1, range of image data

# adversary = AutoAttack(
#     forward_pass, 
#     norm='Linf', 
#     eps=0.01*r, #bound over image domain [0,1]
#     version='custom', 
#     attacks_to_run=['apgd-ce'],
#     verbose=True
# )

# a = adversary.run_standard_evaluation(img, target)

# abs_diff = (a - img).abs()
# print('abs_diff.max', abs_diff.max())

# for i in range(8):
#     display(
#         'orignal',
#         to_pil(img[i].detach().cpu()), 
#         'orignal predict',
#         forward_pass(img[i:i+1]).argmax(),

#         'adversarial',
#         to_pil(a[i].detach().cpu()),
#         'adversarial predict', forward_pass(a[i:i+1]).argmax(),
#         '============='
#     )


In [ ]:
out_dir = '/home/lim38/dataset/imagenet-val-attack'
os.makedirs(out_dir, exist_ok=True)

# subdirs = set([img_and_label[0].split('/')[-2] for img_and_label in dataset.imgs])
# for subdir in subdirs:
#     os.makedirs(f'{out_dir}/{subdir}', exist_ok=True)



In [ ]:
adversary = AutoAttack(
    forward_pass, 
    norm='Linf', 
    eps=5/255, #bound over image domain [0,1]
    version='custom', 
    attacks_to_run=['apgd-ce'],
    verbose=True
)

loader = DataLoader(dataset, batch_size=64, shuffle=False)
i = 0 # global image count
for img, target in tqdm(loader):
    
    img, target = img.cuda(), target.cuda()
    adv = adversary.run_standard_evaluation(img, target)
    
    for a in adv:
        pil = to_pil(a.detach().cpu())
        subdir, fn = dataset.imgs[i][0].split('/')[-2:]
        os.makedirs(f'{out_dir}/{subdir}', exist_ok=True)
        pil.save(f'{out_dir}/{subdir}/{fn}')
        i+=1
